# Signal Generation and Spectrum Analyzer
This example notebooks shows how to use the Synthesis Polyphase Filter Bank to generate multi-tone outputs, and the Analysis PFB as a generic instrument like a spectrum analyzer.

I used this notebook with some external components:
* Noise source: Agilent 346B (with some amplifiers to make it bigger).
* RF splitter/combiner: TB-EP4RKUC+.
* Input: ADC_D
* Output: DAC_B

### Signal Generation
Each channel of the Synthesis PFB has one DDS block connected to it. This DDS is shared and feeds the fine down-conversion that follows the Analysis PFB.
Only one DDS is connected to a specific Synthesis PFB channel. It means that only one tone can be generated on that channel, or none. To generate mulitple output tones, frequencies are spread to cover multiple PFB channels.

### Spectrum Analyzer
For the Spectrum Analyzer functionality, specific channels from the Analysis PFB are selected and their output data samples streamed into Python (numpy) buffers. This functions are all inside the drivers folder if you want to take a look at how that works. To make life easier, convenient functions exist that allow to grab the PFB channel whose center is closer to the specified frequency. The _get_bin_ function does this job.
Two examples are available. The first example will grab all channels to cover a freuency band specified with fstart and fend. Then, the FFT is computed and channels are shown side by side adding the corresponding ADC mixer and channel frequency offset to map into the real-world frequency axis.
The second example shows two channels to emphasize the overlapping structure of the PFB. This second example allows to see more closely how one of the channels exhibits unity gain, while the channel to the right has a slightly lower gain.

In [ ]:
import sys
sys.path.append('./soft')

from pfbs import *

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift

In [ ]:
# Initialize Firmware.
soc = TopSoc('./pfbs_v1.bit')

# Print information.
print(soc)

In [ ]:
################################################
### Initialize Analysis and Synthesis Chains ###
################################################

# Dual Chain: it includes both analysis and synthesis chains.
dual = KidsChain(soc, dual=soc['dual'][0])

# Set ADC/DAC mixer frequency.
dual.set_mixer_frequency(1000)

# Analysis Chain: used as input for spectrum analyzer functionality.
analysis_ch = dual.analysis
analysis_ch.source("input") # by-pass dds product.

# Synthesis Chain: used as output for signal generation.
synthesis_ch = dual.synthesis

In [ ]:
##################################
### Generate some output tones ###
##################################
# Disable all outputs.
synthesis_ch.alloff()

# Set DAC mixer frequency.
#synthesis_ch.set_mixer_frequency(1000)

# Set quantization.
synthesis_ch.qout(2)

# Set tones.
f_v = np.linspace(1100,1110,5)
for f in f_v:
    print("f = {} MHz".format(f))
    synthesis_ch.set_tone_simple(f=f, g=0.5)

In [ ]:
#########################
### Spectrum Analyzer ###
#########################
# Decimation.
analysis_ch.set_decimation(1)
analysis_ch.qout(2)

# Frequency range.
fstart = 1095
fend   = 1115
f = np.arange(fstart, fend, analysis_ch.fc_ch)
print("Spectrum")
print("fstart = {} MHz, fend = {} MHz, fc = {} MHz".format(fstart, fend, analysis_ch.fc_ch))

# Frequency and amplitude vectors.
FF = []
AA = []
plt.figure(dpi=150);
for i,fck in enumerate(f):
    print("i = {}, fck = {} MHz".format(i,fck))
    
    # Transfer data.
    [xi,xq] = analysis_ch.get_bin(fck)
    x = xi + 1j*xq
    
    # Frequency vector.
    F = (np.arange(len(x))/len(x)-0.5)*analysis_ch.fs_ch    
    
    # Normalization factor.
    NF = (2**15)*len(F)

    w = np.hanning(len(x))
    xw = x*w
    YY = fftshift(fft(xw))
    YYlog = 20*np.log10(abs(YY)/NF)
    AA = np.concatenate((AA,YYlog))
    
    Fk = F+fck
    FF = np.concatenate((FF,Fk))
    plt.plot(Fk,YYlog);
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
plt.savefig('spectrum_0.jpg')

In [ ]:
#######################################
### Detail of two neighbor channels ###
#######################################
# Transfer data.
[xi,xq] = analysis_ch.get_bin(1100)
x1 = xi + 1j*xq
[xi,xq] = analysis_ch.get_bin(1101)
x2 = xi + 1j*xq
    
# Spectrum.
F = (np.arange(len(x1))/len(x1)-0.5)*analysis_ch.fs_ch    
w = np.hanning(len(x1))

# Normalization factor.
NF = (2**15)*len(F)

xw1 = x1*w
Y1 = fftshift(fft(xw1))
Y1log = 20*np.log10(abs(Y1)/NF)
xw2 = x2*w
Y2 = fftshift(fft(xw2))
Y2log = 20*np.log10(abs(Y2)/NF)

plt.figure(dpi=150);
plt.plot(F,Y1log,label='channel k');
plt.plot(F+analysis_ch.fc_ch,Y2log,label='channel k+1');
plt.legend();
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
plt.savefig('spectrum_1.jpg')